### Print answers

In [ ]:
import dotenv
dotenv.load_dotenv('/home/oarga/.env')

In [11]:
import os
import json

path_kg = "ontorag copy/src/OntoGen/SACs/GraphRAGqa/answers_GraphRAG_KG.json"
path_kg_onto = "ontorag copy/src/OntoGen/SACs/GraphRAGqa/answers_GraphRAG_KG_Ontology.json"


ANTHROPIC_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "anthropic/claude-3.5-sonnet"
TEMPERATURE = 0

criteria = [
    "Comprehensiveness: How much detail does the answer provide to cover all aspects and details of the question?",
    "Diversity: How varied and rich is the answer in providing different perspectives and insights on the question?",
    "Empowerment: How well does the answer help the reader understand and make informed judgements about the topic?",
    "Directness: How specifically and clearly does the answer address the question?",
]
CRITERIA = (
    3  # 0: Comprehensiveness, 1: Diversity, 2: Empowerment, 3: Directness
)

In [ ]:
data_kg = json.load(open(path_kg))
data_onto = json.load(open(path_kg_onto))

for i in range(len(data_kg)):

    print("=" * 50, " QUESTION ", i, "=" * 50)
    print()
    res = data_kg[i][0].split("\n")
    res = ["\t" + x for x in res if x]
    res = "\n".join(res)
    print(res)
    print()
    print("=" * 50, " ANSWER 1 (GRAPHRAG KG)", "=" * 50)
    print()
    res = data_kg[i][1].split("\n")
    res = ["\t" + x for x in res if x]
    res = "\n".join(res)
    print(res)
    print()
    print("=" * 50, " ANSWER 2 (GRAPHRAG KG + ONTOLOGY)", "=" * 50)
    print()
    res = data_onto[i][1].split("\n")
    res = ["\t" + x for x in res if x]
    res = "\n".join(res)
    print(res)
    print()
    print("*" * 140)
    print("*" * 140)
    print()

In [14]:
import os
from openai import OpenAI

def query_oai(model, prompt, base_url):
    client = OpenAI(
        base_url=base_url,
        api_key=os.getenv("OPENAI_API_KEY")
    )
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=TEMPERATURE,
    )
    return completion.choices[0].message.content

In [ ]:
prompt = """
Given the question and answers below, and given the following criteria: 
{CRITERIA}, 
which of the two answers is better according to the criteria, "ANSWER1" or "ANSWER2"? 
Justify your answer and then answer with " Winner=ANSWER1" or "Winner=ANSWER2"
If both answers are equally good, answer with "Winner=None".

QUESTION: {QUESTION}

========================================= ANSWER 1 =========================================

{ANSWER1}

========================================= ANSWER 2 =========================================

{ANSWER2}
"""

answers = []
for i in range(len(data_kg)):

    print("Processing question ", i)

    question = data_kg[i][0]
    answer1 = data_kg[i][1]
    answer2 = data_onto[i][1]

    formated = prompt.format(
        CRITERIA=criteria[CRITERIA],
        QUESTION=question,
        ANSWER1=answer1,
        ANSWER2=answer2,
    )
    #ans = query_anthropic(client, formated)
    ans = query_oai(MODEL, formated, 'https://openrouter.ai/api/v1')
    if (
        "Winner=ANSWER2".lower() in ans.lower()
        and "Winner=ANSWER1".lower() not in ans.lower()
    ):
        answers.append(2)
    elif (
        "Winner=ANSWER1".lower() in ans.lower()
        and "Winner=ANSWER2".lower() not in ans.lower()
    ):
        answers.append(1)
    else:
        answers.append(0)
    print(formated)
    print(ans)
    print("Answer: ", answers)

winner_kg = [a == 1 for a in answers].count(True)
winner_onto = [a == 2 for a in answers].count(True)
no_winner = [a == 0 for a in answers].count(True)

print("Number of times GraphRAG KG won: ", winner_kg)
print("Number of times GraphRAG KG + Ontology won: ", winner_onto)
print("Number of times there was no clear winner: ", no_winner)